# EvoDiff

In this work, we train and evaluate a series of discrete diffusion models for both unconditional and conditional generation of single protein sequences as well as multiple sequence alignments (MSAs). We test both order-agnostic autoregressive diffusion and discrete denoising diffusion probabilistic models for protein sequence generation; formulate unique, bio-inspired corruption schemes for both classes of models; and evaluate the quality of generated samples for fidelity, diversity, and structural plausibility.

To download and run our code, first open this notebook in a clean conda environment. We recommend creating it with python ```v3.8.5```. You can do so by running ```conda create --name evodiff python=3.8.5```. In that new environment, to download our code, run:

In [ ]:
import sys
!{sys.executable} -m pip install evodiff

You will also need to install PyTorch. We tested our models on `v2.0.1`. Change the below line to install the pytorch version that works for your system.

In [ ]:
conda install pytorch torchvision torchaudio cpuonly -c pytorch

You also need PyTorch Geometric and PyTorch Scatter installed

In [ ]:
conda install pyg -c pyg

In [ ]:
conda install -c conda-forge torch-scatter

## Download model from zenodo

#### Example to download OAAR 38M model


In [4]:
from evodiff.pretrained import OA_AR_38M

checkpoint = OA_AR_38M()
model, collater, tokenizer, scheme = checkpoint

## Unconditional generation

#### To generate one sequence, run:

In [ ]:
from evodiff.generate import generate_oaardm

seq_len = 100
i_sample, i_string = generate_oaardm(model, tokenizer, seq_len, batch_size=1, device='cpu')
print("Generated string:", i_string)

## Conditional generation


#### If you wish to generate a sequence, you can either generate a scaffold structure that supports a desired motif or an IDR. 

In this notebook, we provide the scaffold example. Please view the README in the examples folder for information on generating an IDR. Note that the IDR generation pulls from a database that specifies much of the information you need to manually specify in the scaffold setting.

#### We provide PDB files in the examples folder. You can use the following code segment to visualize the various PDB files and pick one.

In [6]:
# Specify PDB code
pdb = '1prw' 

In [ ]:
!{sys.executable} -m pip install py3Dmol

import py3Dmol

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
view.addModel(open(pdb+'.pdb','r').read(),'pdb')

view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':0.5,'max':0.9}}}) # as color is set to lDDT
# view.setStyle({'cartoon': {'color':'spectrum'}})

view.zoomTo()
view.show()

#### To conditionally generate a sequence, first specify the following arguments (see examples in the README)

In [46]:
# If using cond_task = scaffold, need to specify pdb code and start & end indices for the motif being scaffolded. If defining multiple motifs, supply the start and end -idx motif as a new argument ex: --start-idx 3 --end-idx 10 --start-idx 20 --end-idx 25 (indexes are inclusive of both start and end values).

start_idx = [15, 51]
end_idx = [34, 70]

# WARNING: PDBs are OFTEN indexed at a number that is not 0. If your PDB file begins at 4 and the motif you want to query is residues 5 to 10, as defined by the PDB, your inputs to this code should be --start-idx 1 and --end-idx 6

pdb_code = '1prw'
num_seqs = 3 # Number of sequences generated per scaffold length
scaffold_min = 50 # Minimum scaffold length
scaffold_max = 100 # Maximum scaffold length

data_top_dir = './' # Change this filepath to represent where this notebook exists for you locally

In [ ]:
from evodiff.conditional_generation import generate_scaffold
import pandas as pd
import random

strings = []
start_idxs = []
end_idxs = []
scaffold_lengths = []

for i in range(num_seqs):
    scaffold_length = random.randint(scaffold_min, scaffold_max)

    string, new_start_idx, new_end_idx = generate_scaffold(model, pdb_code, start_idx, end_idx, scaffold_length, data_top_dir, tokenizer, device='cpu')

    strings.append(string)
    start_idxs.append(new_start_idx)
    end_idxs.append(new_end_idx)
    scaffold_lengths.append(scaffold_length)


save_df = pd.DataFrame(list(zip(strings, start_idxs, end_idxs, scaffold_lengths)), columns=['seqs', 'start_idxs', 'end_idxs', 'scaffold_length'])


# The following files are saved to the examples/ folder for ease of viewing the generated sequences

save_df.to_csv('motif_df.csv', index=True)

with open('generated_samples_string.csv', 'w') as f:
    for _s in strings:
        f.write(_s[0]+"\n")
with open('generated_samples_string.fasta', 'w') as f:
    for i, _s in enumerate(strings):
        f.write(">SEQUENCE_" + str(i) + "\n" + str(_s[0]) + "\n")

#### To conditionally generate an MSA, run the following code (see examples in the README)

Note that when conditionally generating an MSA, you can specify query_only = True. By setting this flag to true, you only generate the query sequence. If it is false, then you generate the alignment too.

In [ ]:
from evodiff.pretrained import MSA_OA_AR_MAXSUB
from evodiff.conditional_generation_msa import generate_scaffold_msa

checkpoint = MSA_OA_AR_MAXSUB()
model, collater, tokenizer, scheme = checkpoint

selection_type = 'MaxHamming'
mask_id = checkpoint[2].mask_id
pad_id = checkpoint[2].pad_id

start_idx = [15, 51]
end_idx = [34, 70]

num_seqs = 1 # Number of sequences generated per scaffold length
model_type = msa_oa_ar_maxsub
pdb_code = '1prw'
data_top_dir = './' # Change this filepath to represent where this notebook exists for you locally
query_only_flag = True

strings = []
start_idxs = []
end_idxs = []
seq_lengths = []

for i in range(num_seqs): # no batching
    string, new_start_idx, new_end_idx, seq_len = generate_scaffold_msa(model_type, model, pdb_code, start_idx, end_idx, data_top_dir, tokenizer, query_only = query_only_flag, device='cpu', mask = mask_id, pad = pad_id)

    strings.append(string)
    start_idxs.append(new_start_idx)
    end_idxs.append(new_end_idx)
    seq_lengths.append(seq_len)


save_df = pd.DataFrame(list(zip(strings, start_idxs, end_idxs, seq_lengths)), columns=['seqs', 'start_idxs', 'end_idxs', 'seq_lengths'])
save_df.to_csv('MSA_motif_df.csv', index=True)

with open('MSA_generated_samples_string.csv', 'w') as f:
    for _s in strings:
        f.write(_s[0]+"\n")
with open('MSA_generated_samples_string.fasta', 'w') as f:
    for i, _s in enumerate(strings):
        f.write(">SEQUENCE_" + str(i) + "\n" + str(_s[0]) + "\n")

If you would like to analyze the generated structure by comparing it to the original using the RMSD score, look at the analysis/rmsd_analysis.py script